In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys
os.environ['PYTHONPATH'] = os.environ['PYTHONPATH'] + 'C:\\Users\\rdpuser\\Documents\\Github\\parkit;'
sys.path.append('C:\\Users\\rdpuser\\Documents\\Github\\parkit')

In [3]:
import logging
from parkit import *
logger = logging.getLogger(__name__)

In [4]:
set_default_site('C:\\Users\\rdpuser\\Desktop\\test', create = True)

In [ ]:
# Collection Tests

In [ ]:
get_default_site()

In [ ]:
bind_symbols('module')

In [ ]:
collection_tests(1000, sync = True)

In [5]:
disable_tasks()
compactify()
assert len([e for e in task_executions()]) == 0

In [ ]:
list(task_executions())

In [ ]:
pidtable.get_snapshot()

In [ ]:
enable_tasks()

In [ ]:
set_concurrency(4)

In [ ]:
import random

executions = []
for i in range(16):
    executions.append(collection_tests(1000))
while True:
    assert len(executions) == 16
    if all(e.status in ['crashed', 'failed', 'finished'] for e in executions):
        break
    choice = random.choice([1, 2])
    if choice == 1:
        set_concurrency(random.randint(1, 10))
    elif choice == 2:
        enable_tasks()
assert all(e.result for e in executions)
assert all(e.status == 'finished' for e in executions)
assert all(e.error is None for e in executions)

In [ ]:
[e.status for e in executions]

In [ ]:
get_concurrency()

In [ ]:
disable_tasks()

In [ ]:
[e.status for e in task_executions()]

In [ ]:
# Scheduler Tests

In [ ]:
@task
def foo():
    logger.info('foo')

In [ ]:
enable_tasks()

In [ ]:
schedule(foo, frequency = Frequency.SECOND, period = 1, max_times = 10, start = 'now')

In [ ]:
[s.cancel() for s in schedulers()]

In [ ]:
compactify()

In [ ]:
# File Tests

In [ ]:
f = File()

In [ ]:
f.set_content('hello')

In [ ]:
assert len(f.get_content()) == 5

In [ ]:
assert f.get_content() == 'hello'

In [ ]:
assert f.encoding == 'utf-8'

In [ ]:
assert f.metadata['content-type'] == 'text/plain'

In [ ]:
f.set_content(b'hello')

In [ ]:
assert f.get_content() == b'hello'

In [ ]:
assert isinstance(f.get_content(), bytearray)

In [ ]:
with snapshot(f):
    assert isinstance(f.get_content(zero_copy = True), memoryview)

In [ ]:
assert f.metadata['content-type'] == 'application/octet-stream'

In [ ]:
f.set_content([1, 2, 3])

In [ ]:
assert f.get_content() == [1, 2, 3]

In [ ]:
assert f.metadata['content-type'] == 'application/python-pickle'

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
rows = []
for i in range(1000000):
    rows.append(dict(a = i, b = i))
df = pd.DataFrame(rows)

In [ ]:
f = File(mode = 'wb')

In [ ]:
assert f.size == 0

In [ ]:
f.set_content(df)

In [ ]:
assert df.equals(f.get_content())

In [ ]:
a = np.random.rand(100,100)

In [ ]:
f = File('test', mode = 'wb+')
with f:
    np.save(f, a)

In [ ]:
f.mode = 'rb+'
with f:
    assert np.array_equal(a, np.load(f))

In [ ]:
f = File('test', mode = 'w+')
with f:
    np.savetxt(f, a)

In [ ]:
f.mode = 'r+'
with f:
    assert np.array_equal(a, np.loadtxt(f))

In [ ]:
for obj in directory('__exec__', include_hidden = True):
    print(obj.path)